In [5]:
import sys
print(sys.version)
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import re


3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]


In [ ]:
loader=PyPDFLoader('Neoway.pdf')
text_splitter=RecursiveCharacterTextSplitter(chunk_size=470,chunk_overlap=260)
documents=loader.load()
documents_texts = ""
documents_texts = '\n'.join([doc.page_content[2:] for doc in documents])
documents_texts = re.sub(r'(第[一二三四五六七八九十]+章.*?)(?=\n)', '', documents_texts)
documents_texts = re.sub(r'\n ?(第.{1,5}條(?:之.)*)', r'@\1', documents_texts)
documents_texts = documents_texts.replace('\n', '').replace(' ','').replace('@', ' ')
texts=text_splitter.split_text(documents_texts)
print(documents_texts)

新向系統科技股份有限公司工作規則 第一條（訂立目的）新向系統科技股份有限公司（以下簡稱本公司）為樹立制度，健全組織及管理，特依勞動基準法及相關法令訂定本規則。 第二條（適用範圍）本公司僱用員工，原則均以公開甄選方式為之，並須體檢合格及呈經董事長核定後始得正式任用。凡受本公司僱用從事工作獲致工資之員工均適用之。 第三條（報到手續）新進員工於接到通知後，應依規定之到職日至本公司辦理報到手續，逾期視為自動放棄，該通知因而失其效力，報到時應繳驗下列文件：一、服務誓約書。二、保證書。三、員工調查表。四、繳驗有關證件及國民身分證（核對後發還）。 第四條（勞動契約）本公司因業務需要，僱用員工時，得與員工簽訂定期契約或不定期契約。前項定期契約及不定期契約，依勞動基準法相關規定認定之。 第五條（工作年資計算）員工工作年資之採計方式規定如下：一、定期契約屆滿後或不定期契約因故停止履行後，未滿三個月而訂定新約或繼續履行原約時，員工前後工作年資，應合併計算。二、員工工作年資以服務本公司為限，並自受僱當日起算。三、受本公司調動之工作年資，其年資由本公司續予承認，並應予合併計算。 第六條（新進試用）本公司得與新進員工約定試用，試用期間三個月，但具特殊技能、專長、經歷，經專案簽准者，不在此限。試用合格者依規定正式僱用之。試用期間內考核成績不合格者，須終止契約時，依第七條、第八條、第九條、第十條及第十一條等相關規定辦理，工資發放至停止僱用日為止。 第七條（經預告終止勞動契約）非有下列情事之一者，本公司不得預告員工終止勞動契約：一、歇業或轉讓時。二、虧損或業務緊縮時。三、不可抗力暫停工作在一個月以上時。四、業務性質變更，有減少員工之必要，又無適當工作可供安置時。五、員工對於所擔任之工作確不能勝任時。 第八條（終止契約限制期間之例外）員工在產假期間或職業災害醫療期間，本公司不得終止契約。但若本公司遭天災、事變或其他不可抗力致事業不能繼續者，得報主管機關核定後終止勞動契約。 第九條（資遣預告）依第七條或前條但書規定終止勞動契約時，預告期間如下：一、繼續工作三個月以上一年未滿者，於十日前預告之。二、繼續工作一年以上三年未滿者，於二十日前預告之。三、繼續工作三年以上者，於三十日前預告之。員工於接到前項預告後，為另謀工作，得於工作時間請假外出。其請假時數，每星期不得超過二日之工作時間，請假期間之工資照給。本公

41

In [3]:
embeddings=HuggingFaceEmbeddings(model_name="paraphrase-multilingual-MiniLM-L12-v2")

c:\Users\Public\Anaconda3\envs\breeze2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
if 'db' in locals(): db.delete_collection()
db=Chroma.from_texts(texts,embeddings,persist_directory="db", collection_metadata={"hnsw:space": "cosine"})

In [11]:
import torch
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model = AutoModelForCausalLM.from_pretrained(
    "MediaTek-Research/Breeze-7B-Instruct-v0_1",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2" # optional
)
tokenizer = AutoTokenizer.from_pretrained("MediaTek-Research/Breeze-7B-Instruct-v0_1")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500)
llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.16s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [29]:
retriever = db.as_retriever( search_kwargs={'k': 10})

In [21]:
a = retriever.invoke("員工申訴處理制度是什麼？")
for i in a:
  print(re.findall(r'第.{1,5}條(?:之.)*', i.page_content))
  print(i.page_content)
  print('='*30)

['第一條', '第二條', '第三條', '第四條', '第五條']
新向系統科技股份有限公司工作規則 第一條（訂立目的）新向系統科技股份有限公司（以下簡稱本公司）為樹立制度，健全組織及管理，特依勞動基準法及相關法令訂定本規則。 第二條（適用範圍）本公司僱用員工，原則均以公開甄選方式為之，並須體檢合格及呈經董事長核定後始得正式任用。凡受本公司僱用從事工作獲致工資之員工均適用之。 第三條（報到手續）新進員工於接到通知後，應依規定之到職日至本公司辦理報到手續，逾期視為自動放棄，該通知因而失其效力，報到時應繳驗下列文件：一、服務誓約書。二、保證書。三、員工調查表。四、繳驗有關證件及國民身分證（核對後發還）。 第四條（勞動契約）本公司因業務需要，僱用員工時，得與員工簽訂定期契約或不定期契約。前項定期契約及不定期契約，依勞動基準法相關規定認定之。 第五條（工作年資計算）員工工作年資之採計方式規定如下：一、定期契約屆滿後或不定期契約因故停止履行後，未滿三個月而訂定新約或繼續履行原約時，員工前後工作年資，應合併計算。二、員工工作年資以服務本公司為限，並自受僱當日起算。三、受本公司調動之工作年資，其年資由本公司續予承認，並應予合併計算。
['第五十條', '第五十一條', '第五十二條', '第五十三條', '第五十四條']
第五十條（安全衛生）本公司依法辦理安全衛生工作，員工應遵照相關規定配合辦理。 第五十一條（勞資會議）本公司為協調勞資關係，增進彼此瞭解，促進勞資合作，提高工作效率，依「勞資會議實施辦法」舉辦勞資會議。定期開會並3個月至少召開1次為原則，相互溝通意見，勞雇雙方應本和諧誠信原則，協商解決問題。 第五十二條（員工申訴性騷擾處理制度）員工於工作場所遇有性騷擾時，可向人事單位申訴。申訴專線電話：02-22118317申訴專用傳真：02-22118310申訴專用信箱或電子信箱：alan neoway.com.tw本公司另設有「員工申訴處理制度」。 第五十三條（補充規定）本規則若有法令修改、未盡事宜或涉及員工其他權利義務事項，本公司得視實際需要，按照有關法令規定辦理之。 第五十四條（實施）本規則報請主管機關核備後公告施行，修正時亦同。
['第三條', '第四條', '第五條', '第六條', '第七條、第八條', '第九條、第十條', '第十一條']
第三條（報到手續）新

In [ ]:
import pandas as pd
from langchain_core.prompts import PromptTemplate
template = """你是問答任務的助手。注意:請根據以下文章來回答問題。注意:如果你在文章中找不到答案，請回答"沒有提及"。答案只能從文章內尋找，請勿根據其他資訊來回答。

文章:{context}

問題:{question}

請詳細回答並說明。

回答:"""
custom_rag_prompt = PromptTemplate.from_template(template)


from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)
df = pd.read_excel("Questions_Answers.xlsx")
question = df['Query']
answer = df['標準答案_新向系統']
for (ques,ans) in zip(question,answer):
  print(ques)
  if ques == None:
    break
  print("模型答案:",rag_chain.invoke(ques))
  print("標準答案:",ans)
  print("="*50)

c:\Users\Public\Anaconda3\envs\breeze2\lib\site-packages\transformers\pipelines\base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


工作及休息時間為何?


c:\Users\Public\Anaconda3\envs\breeze2\lib\site-packages\transformers\pipelines\base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


模型答案: 

工作時間：員工正常工作時間每日不超過八小時，每週不得超過四十小時。本公司得視實際需要，依勞動基準法第三十條第二項、第三項及第三十條之一等規定實施彈性工時。

休息時間：員工繼續工作四小時，至少應有三十分鐘之休息。但實行輪班制或其工作有連續性或緊急性者，本公司得在工作時間內，另行調配其休息時間、或由員工依自身需要自行調配其休息時間。

特別休假：員工於本公司繼續工作滿一定期間者，本公司應依下列規定給予特別休假：一、六個月以上一年未滿者，三日。二、一年以上二年未滿者，七日。三、二年以上三年未滿者，十日。四、三年以上五年未滿者，每年十四日。五、五年以上十年未滿者，每年十五日。六、十年以上者，每一年加給一日，加至三十日為止。

例假及休息日：員工每七日中應有二日之休息，其中一日為例假，一日為休息日，工資照給。

休假日：內政部所定應放假之紀念日、節日、勞動節及其他中央主管機關指定應放假日，均應休假，工資照給。

補休：員工於平日或休息日工作後，依其意願選擇補休，並經雇主同意以工作時數1：1換取補休時數；補休期限12個月。補休屆期或契約終止未補休完畢之時數，依當日之工資計算標準發給工資。

休息日工作之工資發給：一、因業務需要，本公司經員工同意於休息日工作者，工作時間在二小時以內者，其工資按平日每小時工資額另再加給一又三分之一以上；工作二小時後再繼續工作者，按平日每小時工資額另再加給一又三分之二以上。二、本公司因天災、事變或突發事件之發生，有使員工於休息日工作之必要者，出勤工資之計算方式，依前款規定計給。
標準答案: 毎日9:00~18:00，中午時段休息一小時。
員工正常工作時間每日不超過八小時，每週不得超過四十小時。員工繼續工作四小時，至少應有三十分鐘之休息。
是否有彈性工作時間?


c:\Users\Public\Anaconda3\envs\breeze2\lib\site-packages\transformers\pipelines\base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

模型答案: 離職預告期限為：

1. 繼續工作三個月以上一年未滿者，於十日前預告之。
2. 繼續工作一年以上三年未滿者，於二十日前預告之。
3. 繼續工作三年以上者，於三十日前預告之。
標準答案: 一、繼續工作三個月以上一年未滿者，於十日前預告之。
二、繼續工作一年以上三年未滿者，於二十日前預告之。
三、繼續工作三年以上者，於三十日前預告之。
是否有離職後競業禁止之約定?


c:\Users\Public\Anaconda3\envs\breeze2\lib\site-packages\transformers\pipelines\base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


模型答案:  沒有提及
標準答案: 沒有提及
員工申訴處理制度是什麼？


c:\Users\Public\Anaconda3\envs\breeze2\lib\site-packages\transformers\pipelines\base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


模型答案: 員工申訴處理制度是員工在工作場所遭遇性騷擾時，可以向公司的人事單位提出申訴。申訴專線電話、申訴專用傳真、申訴專用信箱或電子信箱都提供了申訴的管道。公司另設有「員工申訴處理制度」，以保障員工的權益。
標準答案: 員工於工作場所遇有性騷擾時，可向 人事單位 申訴。
申訴專線電話：02-22118317
申訴專用傳真：02-22118310
申訴專用信箱或電子信箱：alan@neoway.com.tw
本公司是否有勞資會議？
模型答案: 是，本公司有勞資會議。

說明:根據第五十一條的規定，本公司為協調勞資關係、增進彼此瞭解、促進勞資合作、提高工作效率，依「勞資會議實施辦法」舉辦勞資會議。定期開會並3個月至少召開1次為原則，相互溝通意見，勞雇雙方應本和諧誠信原則，協商解決問題。
標準答案: 是
